# SQL Table Information and Discovery

## This NB covers Database Information and Discovery Skills. 

1. Get schema and table information.

2. Get specific table information using PRAGMA.

3. Display sample data from each table.

## We are using a small version of the NYC 311 database that is used in Homework NB9. 

All of the columns from the database will be available, but we have restricted the date range to be a single month of data, in order to be able to reduce the database file size, enabling us to host on Github and manipulate using Google Colab. Note that this database has only a single table.

## We are also using the `university.db` database, which was the data source for the Spring 2025 MT2 exam.

This database has more tables, so it provides additional examples for students to see.

#### The next code cell loads the database into memory. On homework notebooks and exams, you will not have to do this yourself, as the code to load the database will be provided, as we showed in the previous notebooks.

#### Also, the code to load the database below is specific to Google Colab. It is different for notebooks hosted on Vocareum. And again, to reiterate, students WILL NOT be required to write code to load any databases.

In [ ]:
# !wget https://github.com/gt-cse-6040/bootcamp/raw/main/Module%201/Session%204/NYC-311-2M_small.db
# !wget https://github.com/gt-cse-6040/bootcamp/raw/main/Module%201/Session%204/university.db

# create a connection to the database
import sqlite3 as db
import pandas as pd

# Connect to a database (or create one if it doesn't exist)
conn_nyc = db.connect('NYC-311-2M_small.db')
conn_univ = db.connect('university.db')

## What information do we want to know about the database?

1. What tables are in the database?

2. What is the structure of each table (columns and data types)?

3. What does the data look like in each table (data sample)?

*    https://www.sqlite.org/schematab.html

### In SQLite, the table `sqlite_master` contains the metadata about every table in the database.

In SQLite, the sqlite_master table is a system table that contains metadata about the database schema, such as information about tables, indexes, views, and triggers. It's an internal table that SQLite uses to keep track of the structure of the database.

You can query the sqlite_master table to retrieve information about the database schema, including details about the tables in the database, the columns in those tables, and other objects.

Structure of sqlite_master:
The sqlite_master table has the following columns:
*    type: The type of the object (e.g., table, index, view, or trigger).
*    name: The name of the object (e.g., the name of a table, index, or view).
*    tbl_name: The name of the table to which the object belongs (relevant for indexes, views, and triggers).
*    rootpage: The page number of the root b-tree page for the object (relevant for tables and indexes).
*    sql: The SQL statement that was used to create the object (e.g., the CREATE TABLE or CREATE INDEX statement).

**Note that we are using the paradigm that the exams use for exercises.**

In [ ]:
def gettablescema() -> str:
    query = """
            SELECT *
            FROM sqlite_master
            WHERE type='table'
            """
    return query

df_schema_nyc = pd.read_sql(gettablescema(),conn_nyc)
display(df_schema_nyc)

df_schema_univ = pd.read_sql(gettablescema(),conn_univ)
display(df_schema_univ)

### In SQL, the `PRAGMA` statement has many functions (see the documentation).

*    https://www.sqlite.org/pragma.html

In SQLite, `PRAGMA` statements are used to query or modify database settings and retrieve metadata. To get metadata about tables, columns, indexes, and other database objects, SQLite provides specific PRAGMA commands that allow you to extract detailed information about the database schema.

### Here, we are using the `table_info` function, which returns the table structure and column information about the table whose name is passed to it.

This will return the following:
*    cid: Column ID (an integer representing the column's index).
*    name: The name of the column.
*    type: The data type of the column (e.g., INTEGER, TEXT, REAL).
*    notnull: A flag indicating whether the column has a NOT NULL constraint (1 if NOT NULL, 0 if not).
*    dflt_value: The default value for the column (if any).
*    pk: Indicates whether the column is part of the primary key (1 if yes, 0 if no).

#### Note in the function below, we are passing in the table name to send to `PRAGMA table_info()`.

The methodology you see here is how you will want to pass in parameters to your SQL function, that you will include in your query.

In [ ]:
def tablemetadata(tablename: str) -> str:
    
    query = f"""
            PRAGMA table_info('{tablename}')
          """

    return query

# simple example, for the NYC database
df_pragma_nyc = pd.read_sql(tablemetadata('data'),conn_nyc)
display(df_pragma_nyc)

In [ ]:
# more complex example, for multiple tables from the university database

lst_univ_tables = ['student_main','major_crosswalk','scholarship_crosswalk']

for tablename in lst_univ_tables:
    print(f'tablename: {tablename}')
    
    display(pd.read_sql(tablemetadata(tablename),conn_univ))
    
    print('=================')

#### If we did not have the table names, remember that the call to `sqlite_master` returns all of the table names in the database.

So we could loop over the `schema_univ` dataframe, put all of the table names into a list, and call PRAGMA for each. Or just loop over the table names and pass each to PRAGMA.

In [ ]:
lst_univ_tables_full = []

for index, row in df_schema_univ.iterrows():
    lst_univ_tables_full.append(row['tbl_name'])
    
lst_univ_tables_full

In [ ]:
# commented out, to reduce output volume.
# for tablename in lst_univ_tables_full:
#     print(f'tablename: {tablename}')
    
#     display(pd.read_sql(tablemetadata(tablename),conn_univ))
    
#     print('=================')

### Finally, we can do a simple `SELECT * from Table LIMIT #` to get a view on the data itself.

#### Note again that, in the function below, we are passing in the table name and how many row to return.

In [ ]:
def querytables(tablename: str,limit:int=10) -> str:
    query = f"""
            SELECT *
            FROM {tablename}
            LIMIT {limit}
          """
    return query

# simple example, for the NYC database
df_table_nyc = pd.read_sql(querytables('data',5),conn_nyc)
display(df_table_nyc)

In [ ]:
# commented out, to reduce output volume.
# for tablename in lst_univ_tables_full:
#     print(f'tablename: {tablename}')
    
#     display(pd.read_sql(querytables(tablename,5),conn_univ))
    
#     print('=================')

## What are your questions about database discovery?